In [15]:
import pandas as pd
import numpy as np


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam
from datetime import datetime


from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score

In [16]:
#Load dataset into a pandas dataframe
df = pd.read_csv("../data/Cleaned data/data.csv") 

df.dropna(inplace = True)

df.head()

,YEAR,MONTH,DAY,HUMIDITY,WINDSPEED,DATE,TOTALDEMAND,HOLIDAY,MIN,MAX,RAIN,SOLAR,RRP,FORECASTDEMAND,OUTPUT,MONTHDATE,WEEKDAY,WEEKEND,TEMPAVE
0,2016,1,1,0.656341,15.902439,2016-01-01,6853.633437,2.0,15.3,28.6,0.0,32.2,38.472917,6665.366167,23.465,01-2016,4,0,21.95
1,2016,1,2,0.656341,15.902439,2016-01-02,6727.613958,0.0,15.9,26.1,0.0,21.7,36.907292,6236.849955,23.465,01-2016,5,1,21.00
2,2016,1,3,0.688837,14.488372,2016-01-03,6616.406076,0.0,17.5,25.6,0.0,10.3,31.997083,6551.924748,23.465,01-2016,6,1,21.55
3,2016,1,4,0.679545,22.477273,2016-01-04,7367.750278,0.0,18.2,23.6,14.0,6.4,33.424583,6729.993123,23.465,01-2016,0,0,20.90
4,2016,1,5,0.768837,22.581395,2016-01-05,7462.242014,0.0,17.6,20.5,39.0,4.4,33.053958,7333.898202,23.465,01-2016,1,0,19.05


In [17]:

df = df.drop(['MIN','MAX','FORECASTDEMAND', 'MONTHDATE','WEEKEND','RAIN'], axis=1)

df.head()

,YEAR,MONTH,DAY,HUMIDITY,WINDSPEED,DATE,TOTALDEMAND,HOLIDAY,SOLAR,RRP,OUTPUT,WEEKDAY,TEMPAVE
0,2016,1,1,0.656341,15.902439,2016-01-01,6853.633437,2.0,32.2,38.472917,23.465,4,21.95
1,2016,1,2,0.656341,15.902439,2016-01-02,6727.613958,0.0,21.7,36.907292,23.465,5,21.00
2,2016,1,3,0.688837,14.488372,2016-01-03,6616.406076,0.0,10.3,31.997083,23.465,6,21.55
3,2016,1,4,0.679545,22.477273,2016-01-04,7367.750278,0.0,6.4,33.424583,23.465,0,20.90
4,2016,1,5,0.768837,22.581395,2016-01-05,7462.242014,0.0,4.4,33.053958,23.465,1,19.05


In [18]:
# Onehot Encoding for categorial data (Weekday)


# Select the "WEEKDAY" column and create a new dataframe
weekday_df = df[['WEEKDAY']]

# Create a one-hot encoder object

encoder = OneHotEncoder(categories='auto')

# Fit and transform the weekday data
weekday_encoded = encoder.fit_transform(df[['WEEKDAY']]).toarray()

# Create a new dataframe with the encoded weekday data
weekday_df = pd.DataFrame(weekday_encoded, columns=['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN'])

#Drop weekday column
df = df.drop(['WEEKDAY'], axis=1)

# Concatenate the original dataframe with the encoded weekday dataframe
df = pd.concat([df, weekday_df], axis=1)


df.head()

,YEAR,MONTH,DAY,HUMIDITY,WINDSPEED,DATE,TOTALDEMAND,HOLIDAY,SOLAR,RRP,OUTPUT,TEMPAVE,MON,TUE,WED,THU,FRI,SAT,SUN
0,2016,1,1,0.656341,15.902439,2016-01-01,6853.633437,2.0,32.2,38.472917,23.465,21.95,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2016,1,2,0.656341,15.902439,2016-01-02,6727.613958,0.0,21.7,36.907292,23.465,21.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2016,1,3,0.688837,14.488372,2016-01-03,6616.406076,0.0,10.3,31.997083,23.465,21.55,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2016,1,4,0.679545,22.477273,2016-01-04,7367.750278,0.0,6.4,33.424583,23.465,20.90,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,1,5,0.768837,22.581395,2016-01-05,7462.242014,0.0,4.4,33.053958,23.465,19.05,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [19]:


# Convert the 'date' column to a datetime object
df['DATE'] = pd.to_datetime(df['DATE'])

# Set 'DATE' as the index
df.set_index('DATE', inplace=True)




# function to create the sliding window dataset
def create_sliding_window(data, window_size, step_size, target):
    X = []
    y = []
    for i in range(window_size, len(data)):
        X_window = []
        for j in range(0, window_size, step_size):
            X_window.append(data.iloc[i-j-1, :])
        X.append(np.concatenate(X_window, axis=0))
        y.append(data.iloc[i, :]) 
    y2 = [vals[5] for vals in y]
    # display(pd.DataFrame(X))
    return pd.DataFrame(X), pd.DataFrame(y2)

# Define the window size and step size
target = "TOTALDEMAND"
window_size = 1   # 1 day ahead
step_size = 1     # move the window by 1 day at a time

# Apply the sliding window method
X, y = create_sliding_window(df, window_size, step_size, target)
X_all = X.copy() #save for accuracy plotting
X = X.iloc[:, 1:]


In [20]:
# Split the dataset into training and testing sets. 

train_set = (df.index >= datetime(2017, 8, 1)) & (df.index < datetime(2021, 8, 1))
test_set = df.index >= datetime(2021, 8, 1)

test_len = len(test_set)

# identify response variable and predictors
X = df.drop(['TOTALDEMAND'], axis=1).values
y = df['TOTALDEMAND'].values

print(len(train_set))
print(len(test_set))

df.head()

2376
2376


,YEAR,MONTH,DAY,HUMIDITY,WINDSPEED,TOTALDEMAND,HOLIDAY,SOLAR,RRP,OUTPUT,TEMPAVE,MON,TUE,WED,THU,FRI,SAT,SUN
DATE,,,,,,,,,,,,,,,,,,
2016-01-01,2016,1,1,0.656341,15.902439,6853.633437,2.0,32.2,38.472917,23.465,21.95,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-02,2016,1,2,0.656341,15.902439,6727.613958,0.0,21.7,36.907292,23.465,21.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2016-01-03,2016,1,3,0.688837,14.488372,6616.406076,0.0,10.3,31.997083,23.465,21.55,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2016-01-04,2016,1,4,0.679545,22.477273,7367.750278,0.0,6.4,33.424583,23.465,20.90,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-05,2016,1,5,0.768837,22.581395,7462.242014,0.0,4.4,33.053958,23.465,19.05,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Normalize the data


scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X[train_set])
X_test_scaled = scaler.transform(X[test_set])


print(X_train_scaled[:5])
print(len(X_train_scaled))
print(len(X_test_scaled))




[[0.         0.63636364 0.         0.5780574  0.48699955 0.
  0.4068323  0.7116133  0.10793701 0.32412791 0.         1.
  0.         0.         0.         0.         0.        ]
 [0.         0.63636364 0.03333333 0.5780574  0.48699955 0.
  0.40372671 0.64976244 0.10793701 0.3255814  0.         0.
  1.         0.         0.         0.         0.        ]
 [0.         0.63636364 0.06666667 0.69542214 0.28363047 0.
  0.2826087  0.749493   0.10793701 0.39244186 0.         0.
  0.         1.         0.         0.         0.        ]
 [0.         0.63636364 0.1        0.87136016 0.12704281 0.
  0.41304348 0.68692824 0.10793701 0.35901163 0.         0.
  0.         0.         1.         0.         0.        ]
 [0.         0.63636364 0.13333333 0.52090236 0.44853191 0.
  0.42857143 0.55983012 0.10793701 0.34302326 0.         0.
  0.         0.         0.         1.         0.        ]]
1461
337


In [22]:
#Reshape the data for input to the LSTM model
#(This step is fit the data to 3D tensor format for LSTM model to process the sequential data efficiently 
#and capture any temporal dependencies in the data)

#using sliding window approach to create input-output pairs with
#timesteps n = 1



X_train = []
y_train = []
for i in range(1, len(X_train_scaled)):
    X_train.append(X_train_scaled[i-1:i, :])
    y_train.append(X_train_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

X_test = []
y_test = []
for i in range(1, len(X_test_scaled)):
    X_test.append(X_test_scaled[i-1:i, :])
    y_test.append(X_test_scaled[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_train.shape[2]))


print(len(X_train_scaled))
print(len(X_test_scaled))

1461
337


In [ ]:
# NOTE: THIS SECTION OF CODE COULD TAKE SERVERAL HOURS TO RUN TO ACHIEVE THE RESULT
# PLEASE BE MINDFUL, ONLY RUN IF NECESSARY 


# Define the best hyperparameters using GridSearchcv:

def create_model(optimizer = 'adam', neurons=10, dropout=0.2):

    model = Sequential()
    model.add(LSTM(units=neurons, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(units=neurons, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(units=neurons))
    model.add(Dropout(dropout))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
    


# Define the hyperparameters to search over
parameters = {
    'batch_size': [32, 64, 128],
    'epochs': [50, 100, 200],
    'optimizer': [Adam(learning_rate=0.0001), Adam(learning_rate=0.001),Adam(learning_rate=0.01)],
    'neurons': [32, 64, 128],
    'dropout': [0.2, 0.3, 0.4]
}


# Create a KerasRegressor from the defined function
regressor = KerasRegressor(build_fn=create_model, verbose=1)

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=regressor,
                           param_grid=parameters,
                           scoring='neg_mean_squared_error',
                           cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding MSE
print('Best Parameters:', grid_search.best_params_)
print('Best MSE:', -grid_search.best_score_)  



Epoch 1/50
42/42 [==============================] - 4s 3ms/step - loss: 0.3981
Epoch 2/50
42/42 [==============================] - 0s 3ms/step - loss: 0.3784
Epoch 3/50
42/42 [==============================] - 0s 3ms/step - loss: 0.3570
Epoch 4/50
42/42 [==============================] - 0s 3ms/step - loss: 0.3303
Epoch 5/50
42/42 [==============================] - 0s 3ms/step - loss: 0.2949
Epoch 6/50
42/42 [==============================] - 0s 3ms/step - loss: 0.2497
Epoch 7/50
42/42 [==============================] - 0s 3ms/step - loss: 0.1927
Epoch 8/50
42/42 [==============================] - 0s 3ms/step - loss: 0.1307
Epoch 9/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0854
Epoch 10/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0702
Epoch 11/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0673
Epoch 12/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0647
Epoch 13/50
42/42 [==============================

42/42 [==============================] - 0s 3ms/step - loss: 0.3666
Epoch 3/50
42/42 [==============================] - 0s 3ms/step - loss: 0.3434
Epoch 4/50
42/42 [==============================] - 0s 3ms/step - loss: 0.3146
Epoch 5/50
42/42 [==============================] - 0s 3ms/step - loss: 0.2781
Epoch 6/50
42/42 [==============================] - 0s 3ms/step - loss: 0.2323
Epoch 7/50
42/42 [==============================] - 0s 3ms/step - loss: 0.1784
Epoch 8/50
42/42 [==============================] - 0s 3ms/step - loss: 0.1311
Epoch 9/50
42/42 [==============================] - 0s 3ms/step - loss: 0.1003
Epoch 10/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0954
Epoch 11/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0911
Epoch 12/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0858
Epoch 13/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0846
Epoch 14/50
42/42 [==============================] - 0s 3ms

42/42 [==============================] - 0s 3ms/step - loss: 0.3363
Epoch 4/50
42/42 [==============================] - 0s 3ms/step - loss: 0.3131
Epoch 5/50
42/42 [==============================] - 0s 2ms/step - loss: 0.2824
Epoch 6/50
42/42 [==============================] - 0s 2ms/step - loss: 0.2404
Epoch 7/50
42/42 [==============================] - 0s 2ms/step - loss: 0.1868
Epoch 8/50
42/42 [==============================] - 0s 2ms/step - loss: 0.1313
Epoch 9/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0950
Epoch 10/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0845
Epoch 11/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0778
Epoch 12/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0755
Epoch 13/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0709
Epoch 14/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0704
Epoch 15/50
42/42 [==============================] - 0s 2m

42/42 [==============================] - 0s 3ms/step - loss: 0.2789
Epoch 5/50
42/42 [==============================] - 0s 3ms/step - loss: 0.2502
Epoch 6/50
42/42 [==============================] - 0s 3ms/step - loss: 0.2136
Epoch 7/50
42/42 [==============================] - 0s 3ms/step - loss: 0.1710
Epoch 8/50
42/42 [==============================] - 0s 3ms/step - loss: 0.1317
Epoch 9/50
42/42 [==============================] - 0s 3ms/step - loss: 0.1020
Epoch 10/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0929
Epoch 11/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0889
Epoch 12/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0857
Epoch 13/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0814
Epoch 14/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0797
Epoch 15/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0760
Epoch 16/50
42/42 [==============================] - 0s 2

42/42 [==============================] - 0s 2ms/step - loss: 0.2348
Epoch 6/50
42/42 [==============================] - 0s 2ms/step - loss: 0.1988
Epoch 7/50
42/42 [==============================] - 0s 2ms/step - loss: 0.1566
Epoch 8/50
42/42 [==============================] - 0s 2ms/step - loss: 0.1175
Epoch 9/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0969
Epoch 10/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0896
Epoch 11/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0863
Epoch 12/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0856
Epoch 13/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0822
Epoch 14/50
42/42 [==============================] - 0s 2ms/step - loss: 0.0779
Epoch 15/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0760
Epoch 16/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0696
Epoch 17/50
42/42 [==============================] - 0s 

...

42/42 [==============================] - 0s 3ms/step - loss: 0.0037
Epoch 23/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0038
Epoch 24/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0036
Epoch 25/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0034
Epoch 26/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0035
Epoch 27/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 28/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0036
Epoch 29/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 30/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 31/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 32/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 33/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 34/50
42/42 [==============================] -

42/42 [==============================] - 0s 3ms/step - loss: 0.0035
Epoch 24/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0037
Epoch 25/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 26/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 27/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0035
Epoch 28/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 29/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0034
Epoch 30/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 31/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 32/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 33/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 34/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 35/50
42/42 [==============================] -

42/42 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 25/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0034
Epoch 26/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0035
Epoch 27/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 28/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 29/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 30/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0029
Epoch 31/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 32/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0029
Epoch 33/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0029
Epoch 34/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 35/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0028
Epoch 36/50
42/42 [==============================] -

42/42 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 26/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0029
Epoch 27/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 28/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0028
Epoch 29/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0029
Epoch 30/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0027
Epoch 31/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 32/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0028
Epoch 33/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0039
Epoch 34/50
42/42 [==============================] - 0s 4ms/step - loss: 0.0027
Epoch 35/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0027
Epoch 36/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 37/50
42/42 [==============================] -

42/42 [==============================] - 0s 3ms/step - loss: 0.0019
Epoch 27/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0017
Epoch 28/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 29/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0019
Epoch 30/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0017
Epoch 31/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 32/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0017
Epoch 33/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0022
Epoch 34/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0022
Epoch 35/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0014
Epoch 36/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0014
Epoch 37/50
42/42 [==============================] - 0s 3ms/step - loss: 0.0016
Epoch 38/50
42/42 [==============================] -

In [ ]:
# Extract the best hyperparameters
best_batch_size = grid_search.best_params_['batch_size']
best_epochs = grid_search.best_params_['epochs']
best_dropout = grid_search.best_params_['dropout']
best_optimizer = grid_search.best_params_['optimizer']
best_neurons = grid_search.best_params_['neurons']

# Build and compile the model with the best hyperparameters

model = Sequential()
model.add(LSTM(units=best_neurons, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(best_dropout))
model.add(LSTM(units=best_neurons, return_sequences=True))
model.add(Dropout(best_dropout))
model.add(LSTM(units=best_neurons))
model.add(Dropout(best_dropout))
model.add(Dense(units=1, activation='linear'))


# Compile the model with the best optimizer found using grid search
optimizer = best_optimizer
model.compile(loss='mean_squared_error', optimizer=optimizer)


# Train the model with the best number of epochs and batch size found using grid search
history = model.fit(X_train, y_train, epochs=best_epochs, batch_size=best_batch_size, validation_data=(X_test, y_test))



In [ ]:

# Make predictions on the test set
y_pred = model.predict(X_test)


In [ ]:
# Evaluate the LSTM model on the test set
# Calculate MSE and R-squared

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MSE: %.4f' % mse)
print('R-squared: %.4f' % r2)

In [ ]:
# Plot the loss

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import datetime as dt

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['test_loss'], label='validation')
plt.title('Model Loss')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend()
plt.show()


